In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Create a set of 2,000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


Beginning Data Retrieval     
-----------------------------


In [14]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        #time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.description
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Latitude": city_lat,
                          "Longitude": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Percent Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather description": city_weather_desc})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 24 of Set 16 | punta arenas
Processing Record 25 of Set 16 | esperance
Processing Record 26 of Set 16 | hermanus
Processing Record 27 of Set 16 | hasaki
Processing Record 28 of Set 16 | namibe
Processing Record 29 of Set 16 | port alfred
Processing Record 30 of Set 16 | nizhneyansk
City not found. Skipping...
Processing Record 31 of Set 16 | axim
Processing Record 32 of Set 16 | busselton
Processing Record 33 of Set 16 | mataura
Processing Record 34 of Set 16 | avarua
Processing Record 35 of Set 16 | bambous virieux
Processing Record 36 of Set 16 | yerofey pavlovich
Processing Record 37 of Set 16 | nikolskoye
Processing Record 38 of Set 16 | canberra
Processing Record 39 of Set 16 | new norfolk
Processing Record 40 of Set 16 | lumeje
Processing Record 41 of Set 16 | bluff
Processing Record 42 of Set 16 | port elizabeth
Processing Record 43 of Set 16 | arco
Processing Record 44 of Set 16 | san patricio
Processing Record 45 of Set 16 | yellowknife
Processing Record 46 o

In [16]:
city_data_df = pd.DataFrame(city_data)
city_data_df 


,City,Country,Latitude,Longtude,Max Temp,Humidity,Percent Cloudiness,Wind Speed,Weather description,Longitude
0,Punta Arenas,CL,-53.1500,-70.9167,41.11,81,20,25.32,few clouds,NaN
1,Esperance,AU,-33.8667,121.9000,80.44,60,11,17.31,few clouds,NaN
2,Hermanus,ZA,-34.4187,19.2345,59.67,62,60,10.74,broken clouds,NaN
3,Hasaki,JP,35.7333,140.8333,73.80,67,11,5.97,few clouds,NaN
4,Namibe,AO,-15.1961,12.1522,70.00,81,99,7.43,overcast clouds,NaN
...,...,...,...,...,...,...,...,...,...,...
1411,Jacareacanga,BR,-6.2222,NaN,72.48,100,100,0.94,moderate rain,-57.7528
1412,Biltine,TD,14.5333,NaN,72.36,15,21,12.48,few clouds,20.9167
1413,Dingle,PH,10.9995,NaN,89.10,58,73,9.04,broken clouds,122.6711
1414,Berezovyy,RU,51.6667,NaN,27.68,82,89,3.40,overcast clouds,135.7000


In [13]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")